In [ ]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist
import numpy as np
import tqdm
from copy import deepcopy

In [2]:
import collections

class ReplayBuffer():
    def __init__(self, maxlen):
        self.buffer = collections.deque(maxlen=maxlen)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        ind = np.random.choice(len(self.buffer), n, replace=False)
        s, a, r, d, ns = zip(*[self.buffer[i] for i in ind])
        return np.array(s), np.array(a), np.array(r), \
            np.array(d), np.array(ns)
    
    def size(self):
        return len(self.buffer)

In [3]:
class Actor(nn.Module):
    def __init__(self, input_dim=3, hid_dim=128, nlayers=2, out_dim=1, action_bound=[-2, 2]):
        super(Actor, self).__init__()
        self.model = []
        self.out_dim = out_dim
        inp = input_dim
        for i in range(nlayers):
            self.model.append(nn.Linear(inp, hid_dim))
            self.model.append(nn.ReLU())
            inp = hid_dim
        self.model.append(nn.Linear(inp, out_dim))
        self.model = nn.Sequential(*self.model)
        self.action_bound = action_bound

    def forward(self, x):
        a_min, a_max = self.action_bound
        return torch.clamp(self.model(x), a_min, a_max)


class Critic(nn.Module):
    def __init__(self, s_dim=3, a_dim=1, hid_dim=64, nlayers=1, out_dim=1):
        super(Critic, self).__init__()
        self.linear_s = nn.Sequential(
            nn.Linear(s_dim, hid_dim),
            nn.ReLU()
        )
        self.linear_a = nn.Sequential(
            nn.Linear(a_dim, hid_dim),
            nn.ReLU()
        )
        
        self.model = []
        self.out_dim = out_dim
        inp = hid_dim * 2
        for i in range(nlayers):
            self.model.append(nn.Linear(inp, hid_dim))
            self.model.append(nn.ReLU())
            inp = hid_dim
        self.model.append(nn.Linear(inp, out_dim))
        self.model = nn.Sequential(*self.model)

    def forward(self, s, a):
        inp_s = self.linear_s(s)
        inp_a = self.linear_a(a)
        inp = torch.cat([inp_s, inp_a], dim=1)
        return self.model(inp)

In [4]:
class DDPG:
    def __init__(self, mu, mu_optim, q, q_optim, gamma, tau, sigma):
        self.mu = mu
        self.mu_target = deepcopy(mu)
        self.mu_target.eval()
        self.mu_optim = mu_optim
        self.q = q
        self.q_target = deepcopy(q)
        self.q_target.eval()
        self.q_optim = q_optim
        self.gamma = gamma
        self.tau = tau
        self.sigma = sigma
    
    def soft_update(self):
        for p_t, p in zip(self.mu_target.parameters(), self.mu.parameters()):
            p_t.data.copy_(p_t.data * (1 - self.tau) + p.data * self.tau)
        
        for p_t, p in zip(self.q_target.parameters(), self.q.parameters()):
            p_t.data.copy_(p_t.data * (1 - self.tau) + p.data * self.tau)
    
    def learn(self, buffer, batch_size):
        self.mu.train()
        self.q.train()

        s, a, r, d, ns = buffer.sample(batch_size)
        s = torch.from_numpy(s).float()
        a = torch.from_numpy(a).float()[:, None]
        ns = torch.from_numpy(ns).float()
        d = torch.from_numpy(d).float()[:, None]
        r = torch.from_numpy(r).float()[:, None]
        
        target = r + self.gamma * self.q_target(ns, self.mu_target(ns)) * d

        q_loss = F.mse_loss(self.q(s, a), target.detach())
        self.q_optim.zero_grad()
        q_loss.backward()
        self.q_optim.step()
        
        mu_loss = -self.q(s, self.mu(s)).mean()
        self.mu_optim.zero_grad()
        mu_loss.backward()
        self.mu_optim.step()
    
    def sample_action(self, state):
        self.mu.eval()
        state = torch.from_numpy(state)
        with torch.no_grad():
            action = self.mu(state).item()
            action = action + np.random.normal(0, self.sigma)
        return action

In [ ]:
env = gym.make('Pendulum-v1')

In [6]:
n_obs = env.observation_space.shape[0]

In [7]:
action_bound = [env.action_space.low[0], env.action_space.high[0]]

In [ ]:
all_score_list = []
for _ in range(10):
    buffer = ReplayBuffer(maxlen=50000)
    gamma = 0.99
    batch_size = 32
    tau = 5e-3
    sigma = 0.1
    max_episodes = 2500

    actor = Actor(n_obs, 128, 2, 1, action_bound)
    actor_optim = torch.optim.Adam(actor.parameters(), lr=1e-4)
    critic = Critic(n_obs, 1, 64, 1, 1)
    critic_optim = torch.optim.Adam(critic.parameters(), lr=1e-3)
    policy = DDPG(actor, actor_optim, critic, critic_optim, gamma, tau, sigma)

    score_list = []
    for i in tqdm.tqdm(range(max_episodes)):
        s = env.reset()
        score = 0
        done = False
        while not done:
            a = policy.sample_action(s)
            next_s, r, done, info = env.step([a])
            d = 0.0 if done else 1.0
            buffer.put((s, a, r/100, d, next_s))
            s = next_s

            score += r

        if buffer.size() > 2000:
            for k in range(10):
                policy.learn(buffer, batch_size)
                policy.soft_update()

        score_list.append(score)

    all_score_list.append(score_list)